In [ ]:
from stmpy import Driver, Machine

import ipywidgets as widgets
from IPython.display import display

queue = []

class TA:
    def __init__(self):
        print("Init - TA")
        self.button_help = widgets.Button(description = "Help next")
        self.button_help.on_click(self.help_pressed)

        display(self.button_help)

    def show_group_progress(self):
        print("Group progress")

    def help_pressed(self, b):
        if self.button_help.description == "Help next":
            self.button_help.description = "Help finished"
            self.stm.send('help')
        else:
            self.button_help.description = "Help next"
            self.stm.send('help_finished')

    def help_group(self):
        print("help-state")
        
        


#for TA
t0 = {"source": "initial", "target": "waiting"}
t1 = {
    "trigger": "help",
    "source": "waiting",
    "target": "assist",
}
t2 = {
    "trigger": "help_finished",
    "source": "assist",
    "target": "waiting",
}


#TA states
waiting = {'name': 'waiting',
      'entry': 'show_group_progress'
      }
assist = {'name': 'assist',
      'entry': 'help_group'
      }

ta = TA()
ta_machine = Machine(name='stm_ta', transitions=[t0, t1, t2], obj=ta, states=[waiting, assist])
ta.stm = ta_machine

driver = Driver()
driver.add_machine(ta_machine)
driver.start()


#driver.stop()